### Import Required Libraries and Set Up Environment Variables

In [60]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json

In [62]:
# Set environment variables from the .env in the local environment
load_dotenv()

nyt_api_key = os.getenv("lHGnvs1TRa2GVxQHT11dC3BkSoUnAQGw")
tmdb_api_key = os.getenv("f37c9103ee6d8fee93fb96c4bdce03ff")

### Access the New York Times API

In [67]:
# Set the base URL
url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"

# Filter for movie reviews with "love" in the headline
# section_name should be "Movies"
# type_of_material should be "Review"
filter_query = 'section_name:"Movies" AND type_of_material:"Review" AND headline:"love"'

# Use a sort filter, sort by newest
sort = "newest"

# Select the following fields to return:
# headline, web_url, snippet, source, keywords, pub_date, byline, word_count
field_list = "headline,web_url,snippet,source,keywords,pub_date,byline,word_count"

# Search for reviews published between a begin and end date
begin_date = "20130101"
end_date = "20230531"

# Build URL
query_url = "https://api.nytimes.com/svc/movies/v2/reviews/search.json?api-key=lHGnvs1TRa2GVxQHT11dC3BkSoUnAQGw" 


query_url

'https://api.nytimes.com/svc/movies/v2/reviews/search.json?api-key=lHGnvs1TRa2GVxQHT11dC3BkSoUnAQGw'

In [86]:
# Create an empty list to store the reviews
reviews_list = []

# loop through pages 0-19
for page in range(20):
    try:
    # create query with a page number
    # API results show 10 articles at a time
        page_url = query_url + "&offset=" + str(page * 10)
    
    # Make a "GET" request and retrieve the JSON
        response = requests.get(page_url)
        reviews = response.json()
    
    # Add a twelve second interval between queries to stay within API query limits
        time.sleep(12)
    
    # Try and save the reviews to the reviews_list
        try:
            for review in reviews["results"]:
                reviews_list.append(review)
            print(f"Page {page + 1} processed.")
        except:
            print(f"No results found for page {page + 1}.")
            break
    
    except Exception as e:
        print(f"Error fetching data from page {page + 1}: {str(e)}")
        break
print(f"Total reviews fetched: {len(reviews_list)}")

Error fetching data from page 1: Expecting value: line 2 column 1 (char 1)
Total reviews fetched: 0


In [88]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(reviews_list[:5], indent=4))

[]


In [92]:
# Convert reviews_list to a Pandas DataFrame using json_normalize()
reviews_df = pd.json_normalize(reviews_list)
reviews_df

""


In [96]:
# Extract the title from the "headline.main" column and
# save it to a new column "title"
# Title is between unicode characters \u2018 and \u2019. 
# End string should include " Review" to avoid cutting title early
import pandas as pd

# Sample data
data = {
    "headline.main": [
        "Text before \u2018Title to Extract\u2019 Review",
        "Another text \u2018Another Title\u2019 in Review",
        "No Title in this text"
    ]
}

df = pd.DataFrame(data)

def extract_title(text):
    start_index = text.find('\u2018')
    end_index = text.find('\u2019', start_index)
    if start_index != -1 and end_index != -1:
        title = text[start_index + 1:end_index].strip()
        return title + " Review"
    else:
        return ""

df['title'] = df['headline.main'].apply(extract_title)

df


,headline.main,title
0,Text before ‘Title to Extract’ Review,Title to Extract Review
1,Another text ‘Another Title’ in Review,Another Title Review
2,No Title in this text,


In [106]:
def extract_keywords(keyword_list):
    extracted_keywords = ""
    for item in keyword_list:
        keyword = f"{item['name']}: {item['value']}; "
        extracted_keywords += keyword
    return extracted_keywords.strip()  # Remove trailing space

reviews_df['keywords'] = reviews_df['keywords'].apply(extract_keywords)

RangeIndex(start=0, stop=0, step=1)


In [46]:
# Create a list from the "title" column using to_list()
# These titles will be used in the query for The Movie Database
titles_list = df['title'].to_list()
titles_list

['Title to Extract Review', 'Another Title Review', '']

### Access The Movie Database API

In [108]:
# Prepare The Movie Database query
url = "https://api.themoviedb.org/3/search/movie?query="
tmdb_key_string = "&api_key=f37c9103ee6d8fee93fb96c4bdce03ff" 

In [110]:

# Initialize variables
tmdb_movies_list = []
request_counter = 1

# Loop through titles list
for title in titles:
    try:
        # Construct URL for search query
        search_url = url + "&query=" + title
        
        # Perform GET request
        response = requests.get(search_url)
        search_results = response.json()
        
        # Extract movie ID from first result
        movie_id = search_results['results'][0]['id']
        
        # Construct URL for movie query
        movie_url = f"https://api.themoviedb.org/3/movie/{movie_id}?api_key={tmdb_api_key}"
        
        # Perform GET request for movie details
        response = requests.get(movie_url)
        movie_details = response.json()
        
        # Extract required details
        genres = [genre['name'] for genre in movie_details['genres']]
        spoken_languages = [lang['english_name'] for lang in movie_details['spoken_languages']]
        production_countries = [country['name'] for country in movie_details['production_countries']]
        
        # Create dictionary for movie details
        movie_dict = {
            'title': movie_details['title'],
            'original_title': movie_details['original_title'],
            'budget': movie_details['budget'],
            'original_language': movie_details['original_language'],
            'homepage': movie_details['homepage'],
            'overview': movie_details['overview'],
            'popularity': movie_details['popularity'],
            'runtime': movie_details['runtime'],
            'revenue': movie_details['revenue'],
            'release_date': movie_details['release_date'],
            'vote_average': movie_details['vote_average'],
            'vote_count': movie_details['vote_count'],
            'genres': genres,
            'spoken_languages': spoken_languages,
            'production_countries': production_countries
        }
        
        # Append movie_dict to tmdb_movies_list
        tmdb_movies_list.append(movie_dict)
        
        # Print status message
        print(f"Processed {title}: Movie found.")
        
        # Increment request counter
        request_counter += 1
        
        # Sleep for 1 second after every 50 requests
        if request_counter % 50 == 0:
            print("Sleeping for 1 second...")
            time.sleep(1)
    
    except Exception as e:
        print(f"{title}: Movie not found. Error: {str(e)}")

NameError: name 'titles' is not defined

In [112]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(tmdb_movies_list[:5], indent=4))

[]


In [114]:
# Convert the results to a DataFrame
tmdb_df = pd.DataFrame(tmdb_movies_list)

### Merge and Clean the Data for Export

In [118]:
# Merge the New York Times reviews and TMDB DataFrames on title
merged_df = pd.merge(reviews_df, tmdb_df, on='titles')

KeyError: 'titles'

In [15]:
# Remove list brackets and quotation marks on the columns containing lists
# Create a list of the columns that need fixing
columns_to_fix = ['genres', 'spoken_languages', 'production_countries']
characters_to_remove = ["[", "]", "'"]

# Create a list of characters to remove
for column in columns_to_fix:
    merged_df[column] = merged_df[column].astype(str)
    for char in characters_to_remove:
        merged_df[column] = merged_df[column].str.replace(char, '')

# Print head of updated DataFrame
print(merged_df.head())

,title,original_title,budget,genre,language,spoken_languages,homepage,overview,popularity,runtime,...,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,The Attachment Diaries,El apego,0,"Drama, Mystery, Thriller, Horror",es,Spanish,,"Argentina, 1970s. A desperate young woman goes...",0.708,102,...,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
1,You Can Live Forever,You Can Live Forever,0,"Drama, Romance",en,"French, English",https://gooddeedentertainment.com/you-can-live...,"When Jaime, a gay teenager, is sent to live in...",12.027,96,...,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
2,One True Loves,One True Loves,0,"Romance, Drama, Comedy",en,English,,Emma and Jesse are living the perfect life tog...,28.872,100,...,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,None,None,None,By Brandon Yu,"[{'firstname': 'Brandon', 'middlename': None, ...",None
3,The Lost Weekend: A Love Story,The Lost Weekend: A Love Story,0,Documentary,en,,,May Pang lovingly recounts her life in rock & ...,2.368,97,...,‘The Lost Weekend: A Love Story’ Review: When ...,None,None,The Lost Weekend: A Love Story,None,None,None,By Glenn Kenny,"[{'firstname': 'Glenn', 'middlename': None, 'l...",None
4,A Thousand and One,A Thousand and One,0,Drama,en,English,https://www.focusfeatures.com/a-thousand-and-one/,Struggling but unapologetically living on her ...,11.704,116,...,‘A Thousand and One’ Review: A New York Love S...,Critic’s Pick,None,An Unbending Will Meets a Shifting City,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None


In [16]:
# Drop "byline.person" column


In [17]:
# Delete duplicate rows and reset index
merged_df.drop_duplicates(inplace=True)
merged_df.reset_index(drop=True, inplace=True)

,title,original_title,budget,genre,language,spoken_languages,homepage,overview,popularity,runtime,...,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.organization
0,The Attachment Diaries,El apego,0,"Drama, Mystery, Thriller, Horror",es,Spanish,,"Argentina, 1970s. A desperate young woman goes...",0.708,102,...,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,None
1,You Can Live Forever,You Can Live Forever,0,"Drama, Romance",en,"French, English",https://gooddeedentertainment.com/you-can-live...,"When Jaime, a gay teenager, is sent to live in...",12.027,96,...,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,None
2,One True Loves,One True Loves,0,"Romance, Drama, Comedy",en,English,,Emma and Jesse are living the perfect life tog...,28.872,100,...,320,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,None,None,None,By Brandon Yu,None
3,The Lost Weekend: A Love Story,The Lost Weekend: A Love Story,0,Documentary,en,,,May Pang lovingly recounts her life in rock & ...,2.368,97,...,327,‘The Lost Weekend: A Love Story’ Review: When ...,None,None,The Lost Weekend: A Love Story,None,None,None,By Glenn Kenny,None
4,A Thousand and One,A Thousand and One,0,Drama,en,English,https://www.focusfeatures.com/a-thousand-and-one/,Struggling but unapologetically living on her ...,11.704,116,...,971,‘A Thousand and One’ Review: A New York Love S...,Critic’s Pick,None,An Unbending Will Meets a Shifting City,None,None,None,By Manohla Dargis,None


In [18]:
# Export data to CSV without the index
merged_df.to_csv('movie_reviews_tmdb_data.csv', index=False)